In [1]:
import cv2
import numpy as np
from PIL import Image
import multiprocessing as mp

In [71]:
def get_image(path = './cidnei.png'):
    return cv2.imread(path, 0)

In [3]:
def show_image(image):
    img = Image.fromarray(image)
    img.show()

In [41]:
def split_image(image, pieces):
    strips = []
    _, width = image.shape
    strip_size = width // pieces
    
    current_strip = 0
    last_strip = strip_size # 3

    for i in range(pieces):
        if i + 1 == pieces:
            strips.append(image[:, current_strip:])
        else:
            strips.append(image[:, current_strip:last_strip])

        current_strip = last_strip
        last_strip += strip_size
    
    return strips

In [75]:
def convolve2D(image, kernel, padding=0, strides=1):
    kernel = np.flipud(np.fliplr(kernel))

    x_kernel_shape, y_kernel_shape = kernel.shape
    x_image_shape, y_image_shape = image.shape

    x_output = int(((x_image_shape - x_kernel_shape + 2 * padding) / strides) + 1)
    y_output = int(((y_image_shape - y_kernel_shape + 2 * padding) / strides) + 1)
    output = np.zeros((x_output, y_output))

    if padding != 0:
        image_padded = np.zeros((x_image_shape + padding * 2, y_image_shape + padding * 2))
        image_padded[int(padding):int(-1 * padding), int(padding):int(-1 * padding)] = image
    else:
        image_padded = image

    for y in range(y_image_shape):
        if y > y_image_shape - y_kernel_shape:
            break
        if y % strides == 0:
            for x in range(x_image_shape):
                if x > x_image_shape - x_kernel_shape:
                    break
                try:
                    if x % strides == 0:
                        output[x, y] = (kernel * image_padded[x: x + x_kernel_shape, y: y + y_kernel_shape]).sum()
                except:
                    break

    return output

In [76]:
image = get_image()
edge = [[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]]
sharpen = [[0, -1, 0], [-1, 5, -1], [0, -1, 0]]
kernel = np.array(edge)
num_cores = mp.cpu_count()
strips = split_image(image, num_cores)
args = [(strip, kernel) for strip in strips]
pool = mp.Pool(processes=num_cores)
results = pool.starmap(convolve2D, args)
show_image(np.concatenate(results, axis=1))